In [1]:
import pandas
from bs4 import BeautifulSoup
from pathlib import Path
from tqdm.auto import tqdm
import os
import json
from collections import defaultdict

In [2]:
# !~/homebrew/anaconda3/envs/dota/bin/pip install cloudflare-scrape

In [2]:
HTML_DIR = Path('../data/html')

In [3]:
# BASE_INFO_FIELDS = [
#     'адрес', 'год постройки', 'количество этажей', 'тип дома', 'жилых помещений',
#     'серия, тип постройки', 'тип перекрытий', 'материал несущих стен', 'дом признан аварийным',
#     'управляющая компания', 'тип мусоропровода'
# ]
BASE_INFO_FIELDS = ['адрес']

def parse_html(html_path):
    house_info = {}
    with open(html_path, 'r') as html_fh:
        html = html_fh.read()
        soup = BeautifulSoup(html, 'html.parser')
        
        base_info = soup.find_all('dl', class_='dl-horizontal house')[0]
        normalize_f = lambda x: x.text.strip().lower()
        dts = map(normalize_f, base_info.find_all('dt'))
        dds = map(normalize_f, base_info.find_all('dd'))
        for dt, dd in zip(filter(len, dts), filter(len, dds)):
            if dt in BASE_INFO_FIELDS:
                if dt == 'адрес':
                    dd = dd[:-27]
                house_info[dt] = dd
                
    return house_info

parse_html(HTML_DIR / '203928.html')

{'адрес': 'нагатинская наб., 48/2'}

In [4]:
BAD_IDS = [
    1314990, 1348138, 1077171, 1351061,
    1328838, 1360718, 1299658, 1077155,
    1077178, 1299659, 1123058
]

parsed_data = {}
for html in tqdm(os.listdir(HTML_DIR)):
    html_id = int(html[:-5])
    if html_id not in BAD_IDS:
        parsed_data[html_id] = parse_html(HTML_DIR / html)

  0%|          | 0/32727 [00:00<?, ?it/s]

In [5]:
fields = defaultdict(int)
for house_info in parsed_data.values():
    for key in house_info.keys():
        fields[key] += 1
fields

defaultdict(int, {'адрес': 32716})

In [14]:
id_to_address = [{'mingkh_id': html_id, 'mingkh_address': parsed_html['адрес']} for html_id, parsed_html in parsed_data.items()]
len(id_to_address)

32716

In [16]:
with open('../data/mingkh_addresses.json', 'w') as fh:
    json.dump(id_to_address, fh, ensure_ascii=False)